# Super Resolution Network


In [ ]:
import torch
import matplotlib.pyplot as plt

import numpy as np
from PIL import Image
from skimage import metrics

# custom modules
from dataset import Div2kDataset
from utils import ImgTransformer
from models import SResNet
from utils import convert_img

SEED = 42
SCALE = 4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(SEED) 
torch.cuda.manual_seed(SEED)

## Obtain dataset

In [ ]:
!python download.py

## Train model based on configuration

In [ ]:
!python train_sresnet.py

## Load trained model

In [ ]:
model = torch.load('sresnet.pt')['model'].to(DEVICE)
model.train(False)

print('model loaded')

In [ ]:
scale = 4

# get high res image and downscale it 
hr = Image.open('DIV2K/0040.png').crop((200, 500, 1400, 1200))
lr = hr.resize((hr.width // scale, hr.height // scale), Image.BICUBIC)

# try to generate high res image from low res image
sr_tanh = model(convert_img(lr, 'pil', '[0, 1]').unsqueeze(0).to(DEVICE))
sr = convert_img(sr_tanh.detach().squeeze(0), '[-1, 1]', 'pil')

for img, type in [(lr, 'low res'), (hr, 'high res'), (sr, 'super res')]:
    plt.title(type)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

## Metrics

In [ ]:

original_img = np.array(convert_img(hr, 'pil', '[0, 1]'))
superres_img = np.array(convert_img(sr, 'pil', '[0, 1]'))

# value from range (-1, 1)
print('SSIM:', metrics.structural_similarity(original_img, superres_img, channel_axis=0, data_range=1)) 

# higher value means higher similarity (identical img produces zero division)
print('PSNR:', metrics.peak_signal_noise_ratio(original_img, superres_img)) 